In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time

# Selenium WebDriver 설정
s = Service('C:/Users/user/Desktop/ybigta/chromedriver-win32/chromedriver.exe')
driver = webdriver.Chrome(service=s)

# 데이터를 저장할 리스트
data = []

# 1페이지부터 5페이지까지 반복
for page in range(1, 6):
    url = f'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page={page}'
    driver.get(url)

    # 페이지가 완전히 로드될 때까지 대기
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'style-list-item__link')))

    # 페이지의 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 코디맵 아이템들을 찾습니다.
    codimaps = soup.find_all('li', class_='style-list-item')
    detail_links = driver.find_elements(By.CLASS_NAME, 'style-list-item__link')

    for index, codimap in enumerate(codimaps):
        if index * 2 < len(detail_links):
            # 기본 정보 추출
            category = codimap.find('span', class_='style-list-information__text').text.strip()
            title = codimap.find('strong', class_='style-list-information__title').text.strip()
            post_info = codimap.find_all('span', class_='post-information__text')
            date = post_info[0].text.strip()
            views = post_info[1].text.strip().replace('조회 ', '')
            comments = post_info[2].text.strip().replace('댓글 ', '')

            # 상세 페이지로 이동
            detail_link = detail_links[index * 2]
            driver.execute_script("arguments[0].scrollIntoView();", detail_link)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", detail_link)

            # 페이지 로딩 대기
            time.sleep(3)

            # 상세 페이지의 HTML 가져오기
            detail_html = driver.page_source
            detail_soup = BeautifulSoup(detail_html, 'html.parser')

            # 코디 설명, 해시태그, 이미지 URL 추출
            explain = detail_soup.find('p', class_='styling_txt').text.strip()
            hashtags = detail_soup.find_all('a', class_='ui-tag-list__item')
            hashtags_text = [tag.text.strip() for tag in hashtags]
            image_url = detail_soup.find('img', class_='photo')['src']

            # 추출한 정보를 리스트에 추가합니다.
            data.append([category, title, date, views, comments, explain, ', '.join(hashtags_text), image_url])

            # 이전 페이지로 돌아가기
            driver.back()
            time.sleep(3)

# CSV 파일로 저장
with open('musinsa_codimap_data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers', 'codimap_explain', 'codimap_hashtag', 'codimap_imgurl'])
    writer.writerows(data)

# 브라우저 닫기
driver.close()
